####  Desafio de Integração de dados IBGE
##### Julio Cesar Correa 
23/02/2024
 

#### Ingest - Download data from IBGE API 

In [1]:
import os
import pandas as pd, numpy as np
import json
import requests
#pip install import_ipynb
#import import_ipynb


In [2]:
import delta
#from delta import *
#from delta.tables import *
#import mimesis

In [3]:
# Import all functions and types
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [4]:
import pyspark
from pyspark.sql import SparkSession

In [5]:
#read dict path with URLs to hit

In [6]:
def get_spark() -> SparkSession:
    builder = (
        pyspark.sql.SparkSession.builder.appName("IBGE: Data Download")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config(
            "spark.sql.catalog.spark_catalog",
            "org.apache.spark.sql.delta.catalog.DeltaCatalog",
        )
    )
    spark = delta.configure_spark_with_delta_pip(builder).getOrCreate()
    spark.sparkContext.setLogLevel("ERROR")
    return spark

In [7]:
spark = get_spark()

24/02/24 12:37:54 WARN Utils: Your hostname, MacBookJulio.local resolves to a loopback address: 127.0.0.1; using 192.168.5.111 instead (on interface en0)
24/02/24 12:37:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/Users/dal/anaconda3/envs/delta_env/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/dal/.ivy2/cache
The jars for the packages stored in: /Users/dal/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-cda685b2-0e22-4eab-836a-655dc2ad2979;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.2.0 in central
	found io.delta#delta-storage;2.2.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 233ms :: artifacts dl 9ms
	:: modules in use:
	io.delta#delta-core_2.12;2.2.0 from central in [default]
	io.delta#delta-storage;2.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |

24/02/24 12:37:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/02/24 12:37:56 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [8]:
"""
spark = SparkSession.builder.master("local[1]") \
                    .appName('IBGE: Data Download') \
                    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
                    .config( "spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
                    .getOrCreate()
"""

'\nspark = SparkSession.builder.master("local[1]")                     .appName(\'IBGE: Data Download\')                     .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")                     .config( "spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")                     .getOrCreate()\n'

##### Here importing variables, dict from helper file

In [9]:

from helpers import download_endpoint_helper

In [10]:
spark.conf.set("spark.sql.pyspark.inferNestedDictAsStruct.enabled",  True)

In [11]:
#for i in download_endpoint_helper.url_endpoints.values():
#   print(i)

In [12]:
for i in download_endpoint_helper.url_endpoints:
   print(i,':', download_endpoint_helper.url_endpoints[i])
   

url_pop_mun_sp : https://servicodados.ibge.gov.br/api/v3/agregados/9520/periodos/-6/variaveis/93?localidades=N6[N3[35]]
url_localidade_mun_sp : https://servicodados.ibge.gov.br/api/v1/localidades/estados/SP/municipios
url_pib_mun_sp : https://servicodados.ibge.gov.br/api/v3/agregados/5938/periodos/2019|2020|2021/variaveis/37?localidades=N6[N3[35]]
url_idh_uf_sp : https://servicodados.ibge.gov.br/api/v1/pesquisas/37/resultados/33
url_idh_mun_sp : https://servicodados.ibge.gov.br/api/v1/pesquisas/37/resultados/N6[N3[35]]


In [13]:
#print(download_endpoint_helper.url_endpoints["url_localidade_mun_sp"])

##### 1. Python Methods / Helpers

In [14]:
#download API data
def download_api_data(url, data, headers):
    response = None
    try:
        response = requests.get(url, data=body, headers=headers)
    except Exception as e:
        print(e)
    if response != None and response.status_code == 200:
        print(json.dumps(response.url))
        municipios_sp_data = json.loads(response.text)
    return(municipios_sp_data)

In [15]:
#save data as delta table
def save_ibge_data_as_delta(data, path, table_name):
    df = pd.DataFrame(data)
    spark_df = spark.createDataFrame(df.astype(str)) 
    spark_df.withColumn("load_timestamp", current_timestamp())
    spark_df.write.mode(saveMode="overwrite").format("delta").save(path +"/"+ table_name)

In [16]:
# fixed parameters do get data
headers = {'Content-Type': "application/json"}
body = json.dumps({})
page = 1

##### 2. Call and Processing Data

Using a simple loop through url dict values

In [17]:
#fix issue when infer schema
pd.DataFrame.iteritems = pd.DataFrame.items

In [18]:
for idx_key in download_endpoint_helper.url_endpoints:
    target_url = download_endpoint_helper.url_endpoints[idx_key]
    print(idx_key,':', download_endpoint_helper.url_endpoints[idx_key])
    #get the data from API
    response_data = download_api_data(url=target_url, data=body, headers=headers)
    #create path
    path = download_endpoint_helper.delta_root_dir + "/" + download_endpoint_helper.transient_dir 
    #Save Data as Delta format
    save_ibge_data_as_delta(response_data, path, idx_key)

url_pop_mun_sp : https://servicodados.ibge.gov.br/api/v3/agregados/9520/periodos/-6/variaveis/93?localidades=N6[N3[35]]
"https://servicodados.ibge.gov.br/api/v3/agregados/9520/periodos/-6/variaveis/93?localidades=N6%5BN3%5B35%5D%5D"


url_localidade_mun_sp : https://servicodados.ibge.gov.br/api/v1/localidades/estados/SP/municipios
"https://servicodados.ibge.gov.br/api/v1/localidades/estados/SP/municipios"
url_pib_mun_sp : https://servicodados.ibge.gov.br/api/v3/agregados/5938/periodos/2019|2020|2021/variaveis/37?localidades=N6[N3[35]]
"https://servicodados.ibge.gov.br/api/v3/agregados/5938/periodos/2019%7C2020%7C2021/variaveis/37?localidades=N6%5BN3%5B35%5D%5D"
url_idh_uf_sp : https://servicodados.ibge.gov.br/api/v1/pesquisas/37/resultados/33
"https://servicodados.ibge.gov.br/api/v1/pesquisas/37/resultados/33"
url_idh_mun_sp : https://servicodados.ibge.gov.br/api/v1/pesquisas/37/resultados/N6[N3[35]]
"https://servicodados.ibge.gov.br/api/v1/pesquisas/37/resultados/N6%5BN3%5B35%5D%5D"
